<a href="https://colab.research.google.com/github/yiyc-kr/llm-study/blob/main/llama-3-sqlcoder-8b_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 시작하기전에 맨 밑에 accelerate, Seq2SeqTrainingArguments 관련된것부터 해결하고 진행하자. 괜히 컴퓨터 단위 날리지말자...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_path = 'drive/MyDrive/llm-study/'

In [ ]:
# GPU 사용 체크
# TPU를 사용하는 경우 !nvidia-smi 명령어는 작동하지 않습니다.
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 27 13:50:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              48W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# 고용량 메모리 사용 체크
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


데이터 다운로드

In [ ]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [ ]:
import datasets
datasets.__version__ # 2.19.1

'2.19.1'

In [ ]:
from datasets import load_dataset

In [ ]:
en_ko = load_dataset("bongsoo/news_talk_en_ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1299999 [00:00<?, ? examples/s]

In [ ]:
en_ko

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

In [ ]:
en_ko['train']

Dataset({
    features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
    num_rows: 1299999
})

In [ ]:
import pandas as pd

In [ ]:
en_ko_df = pd.DataFrame(en_ko['train'])
en_ko_df.head()

,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
0,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
1,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
2,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
3,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.
4,I can't even consider uninsured treatments.,보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.


In [ ]:
del en_ko

In [ ]:
column_names = ["en", "ko"]
df_forward = pd.DataFrame([en_ko_df.columns], columns=column_names)
en_ko_df.columns = column_names
en_ko_df = pd.concat([df_forward, en_ko_df]).reset_index(drop=True)
en_ko_df.head()

,en,ko
0,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
1,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
2,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
3,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
4,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.


In [ ]:
del df_forward

In [ ]:
len(en_ko_df)

1300000

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(en_ko_df)

In [ ]:
dataset

Dataset({
    features: ['en', 'ko'],
    num_rows: 1300000
})

In [ ]:
# 각 데이터 셋의 샘플 수를 정한다.
num_train = 1200000
num_valid = 90000
num_test = 10000

In [ ]:
en_ko_df_train = en_ko_df.iloc[:num_train]
en_ko_df_valid = en_ko_df.iloc[num_train:num_train+num_valid]
en_ko_df_test = en_ko_df.iloc[-num_test:]

In [ ]:
en_ko_df_train.to_csv(drive_path+"train.tsv", sep='\t', index=False)
en_ko_df_valid.to_csv(drive_path+"valid.tsv", sep='\t', index=False)
en_ko_df_test.to_csv(drive_path+"test.tsv", sep='\t', index=False)

In [ ]:
data_files = {"train": drive_path+"train.tsv", "valid": drive_path+"valid.tsv", "test": drive_path+"test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['en', 'ko'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 10000
    })
})

In [ ]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

('Any academic achievement requires constant repetition.',
 '어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.')

In [ ]:
from datasets import Dataset, DatasetDict

# 기존 dataset을 불러옵니다
dataset = DatasetDict({
    "train": Dataset.from_pandas(dataset['train'].to_pandas()[['ko', 'en']]),
    "valid": Dataset.from_pandas(dataset['valid'].to_pandas()[['ko', 'en']]),
    "test": Dataset.from_pandas(dataset['test'].to_pandas()[['ko', 'en']]),
})

# 결과를 출력하여 확인합니다
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['ko', 'en'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 10000
    })
})


In [ ]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

('Any academic achievement requires constant repetition.',
 '어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.')

In [ ]:
del en_ko_df_train
del en_ko_df_valid
del en_ko_df_test
del en_ko_df

데이터 토큰화

In [ ]:
import torch
torch.__version__ # 2.3.0+cu121

'2.3.0+cu121'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True, legacy=False)

In [ ]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['ko'],
                             text_target=examples['en'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [ ]:
import multiprocessing

In [ ]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

12

In [ ]:
dataset["train"].column_names

['ko', 'en']

In [ ]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU)

Map (num_proc=12):   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
# model_ckpt = "KETI-AIR/ke-t5-base"
# device = "cuda"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
import evaluate
evaluate.__version__ # 0.4.2

'0.4.2'

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
import sacrebleu
sacrebleu.__version__ # 2.4.2

'2.4.2'

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np
np.__version__ # 1.25.2

'1.25.2'

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]/100}

    return result

In [ ]:
!pip uninstall -y accelerate
!pip install git+https://github.com/huggingface/accelerate # 이게 해결법이 되엇다! # 그래도 오류뜸
# 삭제하고 다시 설치해서 세션이 재실행되게끔...
# 아래 Seq2SeqTrainingArguments 보다 먼저 설치 및 import 되어야 할 가능성이 있다...

Found existing installation: accelerate 0.31.0.dev0
Uninstalling accelerate-0.31.0.dev0:
  Successfully uninstalled accelerate-0.31.0.dev0
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-bit6ymnb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-bit6ymnb
  Resolved https://github.com/huggingface/accelerate to commit c3f422699a5cb7665edd5420b8b106512c09a85d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.31.0.dev0-py3-none-any.whl size=308482 sha256=0b190bf4d2417fbb907e2e48f557bed38222278e92d7e2f8b7aaafdcbb51a772
  Stored in directory: /tmp/pip-ephem-wheel-cache-8hzt2jkg/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
import accelerate
accelerate.__version__ # 0.30.1.dev0 다시 재설치해도 왼쪽버전으로...

'0.30.1.dev0'

In [ ]:
from transformers import EarlyStoppingCallback, Seq2SeqTrainingArguments

In [ ]:
import transformers
transformers.__version__ # 4.41.0

'4.41.0'

In [ ]:
# !pip install transformers[torch]

In [ ]:
# 새로운 TrainingArguments 설정
training_args = Seq2SeqTrainingArguments(
    output_dir=drive_path+"ko_en_chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=1,  # 에폭 수 조절
    # save_steps=500,
    # save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

In [ ]:
# 기존 체크포인트 경로
# checkpoint = "chkpt/checkpoint-9000"

# 새로운 Trainer 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],  # 조기 종료 추가
)

# 기존 체크포인트에서 학습 재개
# trainer.train(resume_from_checkpoint=checkpoint)

# 그냥 새로 학습
# trainer.train()

In [ ]:
# !pip install transformers[torch]

In [ ]:
# !pip install --upgrade transformers[torch]
# !pip install --upgrade accelerate

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("./results")

In [ ]:
!tar -czvf results.tar.gz results/

results/
results/tokenizer.json
results/spiece.model
results/model.safetensors
results/tokenizer_config.json
results/config.json
results/generation_config.json
results/special_tokens_map.json
results/training_args.bin


In [ ]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
input_text = [
    "I got my peaches out in Georgia (oh, yeah, shit)",
    "I'm hungry. But I won't eat. Because I'm on a diet. I have to exercise, but it's annoying. I just want to sleep."
]

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

In [ ]:
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

['<pad> 나는 조지아에서 복숭아를 땄어요(오, 맞아요, <unk>).</s><pad><pad><pad><pad><pad><pad>',
 '<pad> 저는 배가 <unk>지만 먹지 않을 것이에요, 다이어트가 돼서 운동해야 하는데 귀찮고, 그냥 자고 싶어.</s>']

In [ ]:
new_trainer.save_model("./new_results")